<a href="https://colab.research.google.com/github/xdomodomox/colab/blob/main/%E5%A4%A9%E7%9C%9F%E9%A0%90%E6%B8%AC%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# 讀取股票數據
data = pd.read_csv('/content/drive/MyDrive/ANN_Timeseries_Forecast_python_demo/Data/Close.csv')

# 提取 Close 數據
close_data = data['Close'].values

split =int(0.8*data.shape[0])
Train,Test  = close_data[0:split] , close_data[split:]

# 擷取最後一天的股價
last_price = Train[-1]

# 使用天真預測法預測未來股價
future_price = np.full_like(Test, last_price)

# 計算均方根誤差(RMSE)作為基準值的損失函數
baseline_rmse = np.sqrt(mean_squared_error(Test, future_price))

print('基準值的RMSE:', baseline_rmse)


基準值的RMSE: 247.42631655283768
